In [21]:
filepath = r"C:\Users\HP\Downloads\calgary_access_log"
with open(filepath, "rt", encoding = "utf-8", errors = "ignore") as logs:
    for i in range(5):
        print(logs.readline())

local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150

local - - [24/Oct/1994:13:41:41 -0600] "GET 1.gif HTTP/1.0" 200 1210

local - - [24/Oct/1994:13:43:13 -0600] "GET index.html HTTP/1.0" 200 3185

local - - [24/Oct/1994:13:43:14 -0600] "GET 2.gif HTTP/1.0" 200 2555

local - - [24/Oct/1994:13:43:15 -0600] "GET 3.gif HTTP/1.0" 200 36403



In [24]:
import pandas as pd
import re
import os
import sys
from datetime import datetime

filepath = r"C:\Users\HP\Downloads\calgary_access_log"

# Checking file availability
if not os.path.isfile(filepath):
    raise FileNotFoundError (f"file not found {filepath}")

# regex pattern
log_pattern = re.compile(
    r'(?P<host>\S+) '                
    r'\S+ \S+ '                       
    r'\[(?P<timestamp>[^\]]+)\] '     
    r'"(?P<Request>[^"]+)" '         
    r'(?P<Status>\d{3}) '            
    r'(?P<size>\S+)'                 
)


# Fucntion to parse line using regex

def parse_line(line):
    match = log_pattern.match(line)
    if match:
        return match.groupdict()
    else:
        return None
        
valid_entry = []
invalid_entry = []

# Exception handling try/Except block
try:
    with open(filepath, 'rt', encoding = 'utf-8', errors = 'ignore') as logs:
       for line_number , line in enumerate(logs,1):
           parsed = parse_line(line) #Calling the function
           if parsed:
               valid_entry.append(parsed)
           else:
               invalid_entry.append({'line_number': line_number, 'raw_line': line.strip()})

except FileNotFoundError:
      print(f"File {filepath} does not exist.")
      sys.exit(1)

except Exception as e:
      print(f"An unexpected error occured during file reading {e}")
      sys.exit(1)

# flag invalid log_entries

df = pd.DataFrame(valid_entry)
df_invalid = pd.DataFrame(invalid_entry)

print(f"Valid Log Entries {len(df)}")
print(f"Invalid Log Entries {len(df_invalid)}")


print(df.head())


Valid Log Entries 724836
Invalid Log Entries 1903
    host                   timestamp                  Request Status   size
0  local  24/Oct/1994:13:41:41 -0600  GET index.html HTTP/1.0    200    150
1  local  24/Oct/1994:13:41:41 -0600       GET 1.gif HTTP/1.0    200   1210
2  local  24/Oct/1994:13:43:13 -0600  GET index.html HTTP/1.0    200   3185
3  local  24/Oct/1994:13:43:14 -0600       GET 2.gif HTTP/1.0    200   2555
4  local  24/Oct/1994:13:43:15 -0600       GET 3.gif HTTP/1.0    200  36403


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724836 entries, 0 to 724835
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   host       724836 non-null  object
 1   timestamp  724836 non-null  object
 2   Request    724836 non-null  object
 3   Status     724836 non-null  object
 4   size       724836 non-null  object
dtypes: object(5)
memory usage: 27.7+ MB


In [26]:
df.tail()

,host,timestamp,Request,Status,size
724831,local,11/Oct/1995:14:11:39 -0600,GET 2.gif HTTP/1.0,200,2555
724832,local,11/Oct/1995:14:11:40 -0600,GET 3.gif HTTP/1.0,200,36403
724833,local,11/Oct/1995:14:11:45 -0600,GET 11459.html HTTP/1.0,404,-
724834,local,11/Oct/1995:14:11:58 -0600,GET 11459.html HTTP/1.0,404,-
724835,local,11/Oct/1995:14:14:17 -0600,GET index.html HTTP/1.0,304,0


In [27]:
## Cleaning the column size setting '-' to 0
df['size'] = df['size'].replace('-', 0).astype(int)

In [28]:
df.tail()

,host,timestamp,Request,Status,size
724831,local,11/Oct/1995:14:11:39 -0600,GET 2.gif HTTP/1.0,200,2555
724832,local,11/Oct/1995:14:11:40 -0600,GET 3.gif HTTP/1.0,200,36403
724833,local,11/Oct/1995:14:11:45 -0600,GET 11459.html HTTP/1.0,404,0
724834,local,11/Oct/1995:14:11:58 -0600,GET 11459.html HTTP/1.0,404,0
724835,local,11/Oct/1995:14:14:17 -0600,GET index.html HTTP/1.0,304,0


In [29]:
##Parsing timestamp string to datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'], format = '%d/%b/%Y:%H:%M:%S %z', errors = 'coerce', utc =True)

In [31]:
# Extracting filename from request column
df['method'] = df['Request'].str.split().str[0]
df['filename'] = df['Request'].str.split().str[1]
df['protocol'] = df['Request'].str.split().str[2]



In [35]:
# Extract fileextension from filename 
df['file_extension'] = df['filename'].str.extract(r'(\.[a-zA-Z0-9]+)$', expand = False).fillna('')

In [37]:
#Dropping the original request column
df.drop(columns = ['Request'], inplace = True)
print(df.head())

    host                 timestamp Status   size method    filename  protocol  \
0  local 1994-10-24 19:41:41+00:00    200    150    GET  index.html  HTTP/1.0   
1  local 1994-10-24 19:41:41+00:00    200   1210    GET       1.gif  HTTP/1.0   
2  local 1994-10-24 19:43:13+00:00    200   3185    GET  index.html  HTTP/1.0   
3  local 1994-10-24 19:43:14+00:00    200   2555    GET       2.gif  HTTP/1.0   
4  local 1994-10-24 19:43:15+00:00    200  36403    GET       3.gif  HTTP/1.0   

  file_extension  
0          .html  
1           .gif  
2          .html  
3           .gif  
4           .gif  


In [38]:
## Converting data type of status to int
df['Status'] = df['Status'].astype(int)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724836 entries, 0 to 724835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   host            724836 non-null  object             
 1   timestamp       724836 non-null  datetime64[ns, UTC]
 2   Status          724836 non-null  int32              
 3   size            724836 non-null  int32              
 4   method          724836 non-null  object             
 5   filename        724836 non-null  object             
 6   protocol        723263 non-null  object             
 7   file_extension  724836 non-null  object             
dtypes: datetime64[ns, UTC](1), int32(2), object(5)
memory usage: 38.7+ MB


In [40]:
df.tail()

,host,timestamp,Status,size,method,filename,protocol,file_extension
724831,local,1995-10-11 20:11:39+00:00,200,2555,GET,2.gif,HTTP/1.0,.gif
724832,local,1995-10-11 20:11:40+00:00,200,36403,GET,3.gif,HTTP/1.0,.gif
724833,local,1995-10-11 20:11:45+00:00,404,0,GET,11459.html,HTTP/1.0,.html
724834,local,1995-10-11 20:11:58+00:00,404,0,GET,11459.html,HTTP/1.0,.html
724835,local,1995-10-11 20:14:17+00:00,304,0,GET,index.html,HTTP/1.0,.html


In [44]:
# Q1 Total number of HTTP requests
total_request = len(df)
print('No_of_Request :', total_request)

No_of_Request : 724836


In [45]:
# Q2 Number of unique hosts
unique_hosts = df['host'].nunique()
print('No_of_unique_host :', unique_hosts)

No_of_unique_host : 2


In [48]:
# Q3 Date-wise unique filename counts
date_unique_files = df.groupby('timestamp')['filename'].nunique().to_dict()
print('unique_filename_count_datewise', list(date_unique_files.items())[:5])

unique_filename_count_datewise [(Timestamp('1994-10-24 19:41:41+0000', tz='UTC'), 2), (Timestamp('1994-10-24 19:43:13+0000', tz='UTC'), 1), (Timestamp('1994-10-24 19:43:14+0000', tz='UTC'), 1), (Timestamp('1994-10-24 19:43:15+0000', tz='UTC'), 1), (Timestamp('1994-10-24 19:43:17+0000', tz='UTC'), 1)]


In [51]:
# Q4 Number of 404 response codes
count_404 = (df['Status'] == 404).sum()
print('404_response_count :', count_404)

404_response_count : 23517


In [53]:
# Q5 Top 15 filenames with 404 responses
top_404_files = (
    df[df['Status'] == 404]
    .groupby('filename')
    .size()
    .sort_values(ascending =False)
    .head(15)
    .reset_index(name = 'count')
)

print('top_404_files :' , list(zip(top_404_files['filename'], top_404_files['count'])))


top_404_files : [('index.html', 4737), ('4115.html', 902), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 259), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('40.html', 148), ('3414.gif', 148), ('488.gif', 148), ('9678.gif', 142)]


In [55]:
# Q6 Top 15 file extensions with 404 responses
top_404_files = (
    df[df['Status'] == 404]
    .groupby('file_extension')
    .size()
    .sort_values(ascending =False)
    .head(15)
    .reset_index(name = 'count')
)

print('top_404_extension :' , list(zip(top_404_files['file_extension'], top_404_files['count'])))

top_404_extension : [('.html', 12213), ('.gif', 7213), ('.xbm', 826), ('.ps', 754), ('.jpg', 527), ('', 505), ('.txt', 497), ('.GIF', 135), ('.htm', 109), ('.cgi', 77), ('.com', 46), ('.Z', 41), ('.dvi', 40), ('.ca', 36), ('.hmtl', 30)]


In [57]:
# Q7 Total bandwidth per day for july 1995
df['date_Str'] = df['timestamp'].dt.strftime('%Y-%m-%d') #extracting date from timestamp
df['date_Str'].head()

0    1994-10-24
1    1994-10-24
2    1994-10-24
3    1994-10-24
4    1994-10-24
Name: date_Str, dtype: object

In [60]:
july_1995 = df[(df['timestamp'].dt.year == 1995) & (df['timestamp'].dt.month == 7) & (df['size']>0)]
bandwidth_per_Day = july_1995.groupby('date_Str')['size'].sum().to_dict()
print('Total_bandwidth_july_1995 :', list(bandwidth_per_Day.items())[:5])

Total_bandwidth_july_1995 : [('1995-07-01', 17002716), ('1995-07-02', 7895368), ('1995-07-03', 11741216), ('1995-07-04', 25070338), ('1995-07-05', 22468066)]


In [62]:
# Q8 Hourly request distribution within 24 hours
df['hour'] = df['timestamp'].dt.hour
hourly_requests = df.groupby('hour').size().to_dict()
print('Hourly_request_distribution', hourly_requests)

Hourly_request_distribution {0: 39610, 1: 32685, 2: 30736, 3: 28168, 4: 26027, 5: 22853, 6: 19868, 7: 17079, 8: 13879, 9: 11438, 10: 10574, 11: 10432, 12: 12319, 13: 15191, 14: 22093, 15: 30921, 16: 38019, 17: 46323, 18: 45768, 19: 50058, 20: 51160, 21: 52919, 22: 50514, 23: 46202}


In [63]:
# Q9 Top 10 most requested filenames
top_files = (
    df.groupby('filename')
    .size()
    .sort_values(ascending = False)
    .head(10)
    .reset_index(name='count')
)
print('Top_requested_filenames', list(zip(top_files['filename'], top_files['count'])))

Top_requested_filenames [('index.html', 140074), ('3.gif', 24006), ('2.gif', 23606), ('4.gif', 8018), ('244.gif', 5149), ('5.html', 5010), ('4097.gif', 4874), ('8870.jpg', 4493), ('6733.gif', 4278), ('8472.gif', 3843)]


In [65]:
# Q10 HttP response code distribution
status_counts = df['Status'].value_counts().to_dict()
print('HTTP_code_distribution :' , status_counts)

HTTP_code_distribution : {200: 568345, 304: 97792, 302: 30295, 404: 23517, 403: 4741, 401: 46, 501: 43, 500: 42, 400: 15}
